# Experiment with Argopy from PR575 on Argo Vocabulary

<img src="https://raw.githubusercontent.com/euroargodev/argopy/master/docs/_static/argopy_logo_long.png" alt="argopy logo" width="200"/>

If you don't know where to start, have a look to the [PR description (↗)](https://github.com/euroargodev/argopy/pull/575).

Rq: This notebook is intended to be executed from a Binder instance for demonstration purposes

### Set-up and import

⚠️ Execute the following cell to ensure you'll be able to import new Argopy features.

In [1]:
import os
import subprocess
if os.getenv('BINDER_SERVICE_HOST'):
    print("⚠️ Running on a Binder instance, be aware that your changes won't be saved")
    try:
        from argopy import ArgoReferenceTable
        print("✅ You're all set to play with new Argopy features !")
    except ImportError:
        print("⚙️ We first need to install Argopy")
        print("Installing... (kernel will restart at the end of the process)")
        subprocess.run(["pip", "install", "/home/jovyan/argopy"], stdout=subprocess.DEVNULL) # install argopy from this PR branch
        os._exit(00)  # re-start jupyter kernel

## ArgoReferenceTable

Rq: I hesitated to call this class ``ArgoReferenceVocabulary`` or ``ArgoVocabulary``. But since the Argo user's manual uses "Reference Table", I opted to keep this approach and do not stick too much to the NVS jargon (which is still visible in the internal machinery though).

The ``ArgoReferenceTable`` instance holds all the Argo referencing system information:

- the comprehensive logic (use ``SENSOR`` in place of ``R25``),
- the table meta-data are in read-only attributes, e.g. ``art.description`` and ``art.version``, ``art.date``, etc...,
- the list of table values/rows, which are accessible through label-based indexing, e.g. ``art['CTD_TEMP_CNDC']``; this means that the instance can be used almost like a dictionary (e.g. for inclusion assertion and iteration),
- values export/search methods, e.g. ``art.search()``, ``art.to_dataframe()``,  ``art.to_dict()``


### Creation

In [2]:
from argopy import ArgoReferenceTable

# Use an Argo parameter name, documented by one of the Argo reference tables:
art = ArgoReferenceTable('SENSOR')

# or a reference table identifier:
art = ArgoReferenceTable('R25')

# or a URN:
art = ArgoReferenceTable.from_urn('SDN:R25::CTD_TEMP')

art

<argo.reference.table> 'R25'/'SENSOR'
long_name: "Argo sensor types"
description: "Terms describing sensor types mounted on Argo floats. Argo netCDF variable SENSOR is populated by R25 altLabel."
uri: http://vocab.nerc.ac.uk/collection/R25/current/
version: 4 (2025-08-29 03:00:01)
keys[33]: 'ACOUSTIC', 'ACOUSTIC_GEOLOCATION', 'BACKSCATTERINGMETER_BBP470', 'BACKSCATTERINGMETER_BBP532', 'BACKSCATTERINGMETER_BBP700', 'BACKSCATTERINGMETER_TURBIDITY', 'CTD_CNDC', 'CTD_PRES', 'CTD_TEMP', 'CTD_TEMP_CNDC' and more ...

### Attributes


In [3]:
# All possible attributes are listed in:
art.attrs

('identifier',
 'parameter',
 'long_name',
 'description',
 'version',
 'date',
 'uri',
 'nvs')

In [4]:
# Reference Table attributes:
art.parameter   # Name of the netcdf dataset parameter filled with values from this table
art.identifier  # Reference Table ID
art.description # [nvs['@graph']['@type']=='skos:Collection']["dc:description"]
art.uri         # [nvs['@graph']['@type']=='skos:Collection']["@id"]
art.version     # [nvs['@graph']['@type']=='skos:Collection']['owl:versionInfo']
art.date        # [nvs['@graph']['@type']=='skos:Collection']['dc:date']

Timestamp('2025-08-29 03:00:01')

In [5]:
# Raw NVS json data:
art.nvs.keys()

dict_keys(['@graph', '@context'])

### Indexing and values

In [6]:
art = ArgoReferenceTable('SENSOR')
art

<argo.reference.table> 'R25'/'SENSOR'
long_name: "Argo sensor types"
description: "Terms describing sensor types mounted on Argo floats. Argo netCDF variable SENSOR is populated by R25 altLabel."
uri: http://vocab.nerc.ac.uk/collection/R25/current/
version: 4 (2025-08-29 03:00:01)
keys[33]: 'ACOUSTIC', 'ACOUSTIC_GEOLOCATION', 'BACKSCATTERINGMETER_BBP470', 'BACKSCATTERINGMETER_BBP532', 'BACKSCATTERINGMETER_BBP700', 'BACKSCATTERINGMETER_TURBIDITY', 'CTD_CNDC', 'CTD_PRES', 'CTD_TEMP', 'CTD_TEMP_CNDC' and more ...

In [7]:
# Values (or concept) within this reference table:
art.keys()   # List of reference values name
art.values() # List of :class:`ArgoReferenceValue`
len(art)     # Number of reference values

33

In [8]:
# Check for values:
'CTD_TEMP_CNDC' in art  # Return True

True

In [9]:
# Index by value key, like a simple dictionary:
art['CTD_TEMP_CNDC']  # Return a :class:`ArgoReferenceValue` instance

<argo.reference.table.value> 'CTD_TEMP_CNDC'
long_name: "Temperature sensor for conductivity on a CTD"
definition: "Specific temperature sensor used with conductivity to calculate salinity on a CTD"
urn: "SDN:R25::CTD_TEMP_CNDC"
uri: http://vocab.nerc.ac.uk/collection/R25/current/CTD_TEMP_CNDC/
version: 1 (2023-11-24 09:35:13)
deprecated: False
reference/parameter: R25/SENSOR
relations[6]:
  - "related" to 6 values : 'R27/RBR_ARGO3', 'R27/RBR_ARGO3_DEEP4K', 'R27/RBR_ARGO', 'R27/RBR_ARGO3_DEEP6K', 'R27/RBR' and 'R27/RBR_CTD'
context: (not loaded yet, use key indexing to load)
extra: (no extra attributes from definition string)

In [10]:
# Allows to iterate over all values/concepts:
for concept in art:
    print(f"{concept.name:30s}= {concept.urn}")

ACOUSTIC                      = SDN:R25::ACOUSTIC
ACOUSTIC_GEOLOCATION          = SDN:R25::ACOUSTIC_GEOLOCATION
BACKSCATTERINGMETER_BBP470    = SDN:R25::BACKSCATTERINGMETER_BBP470
BACKSCATTERINGMETER_BBP532    = SDN:R25::BACKSCATTERINGMETER_BBP532
BACKSCATTERINGMETER_BBP700    = SDN:R25::BACKSCATTERINGMETER_BBP700
BACKSCATTERINGMETER_TURBIDITY = SDN:R25::BACKSCATTERINGMETER_TURBIDITY
CTD_CNDC                      = SDN:R25::CTD_CNDC
CTD_PRES                      = SDN:R25::CTD_PRES
CTD_TEMP                      = SDN:R25::CTD_TEMP
CTD_TEMP_CNDC                 = SDN:R25::CTD_TEMP_CNDC
EM                            = SDN:R25::EM
FLOATCLOCK_MTIME              = SDN:R25::FLOATCLOCK_MTIME
FLUOROMETER_CDOM              = SDN:R25::FLUOROMETER_CDOM
FLUOROMETER_CHLA              = SDN:R25::FLUOROMETER_CHLA
IDO_DOXY                      = SDN:R25::IDO_DOXY
OPTODE_DOXY                   = SDN:R25::OPTODE_DOXY
RADIOMETER_DOWN_IRR           = SDN:R25::RADIOMETER_DOWN_IRR
RADIOMETER_DOWN_IRR380    

### Export methods

In [11]:
art = ArgoReferenceTable('SENSOR')
art

<argo.reference.table> 'R25'/'SENSOR'
long_name: "Argo sensor types"
description: "Terms describing sensor types mounted on Argo floats. Argo netCDF variable SENSOR is populated by R25 altLabel."
uri: http://vocab.nerc.ac.uk/collection/R25/current/
version: 4 (2025-08-29 03:00:01)
keys[33]: 'ACOUSTIC', 'ACOUSTIC_GEOLOCATION', 'BACKSCATTERINGMETER_BBP470', 'BACKSCATTERINGMETER_BBP532', 'BACKSCATTERINGMETER_BBP700', 'BACKSCATTERINGMETER_TURBIDITY', 'CTD_CNDC', 'CTD_PRES', 'CTD_TEMP', 'CTD_TEMP_CNDC' and more ...

In [12]:
# Export table attributes to a dictionary (rq: this is not to export values):
art.to_dict()

{'identifier': 'R25',
 'parameter': 'SENSOR',
 'long_name': 'Argo sensor types',
 'description': 'Terms describing sensor types mounted on Argo floats. Argo netCDF variable SENSOR is populated by R25 altLabel.',
 'version': '4',
 'date': Timestamp('2025-08-29 03:00:01'),
 'uri': 'http://vocab.nerc.ac.uk/collection/R25/current/'}

In [13]:
art.to_dict(keys=['parameter', 'date', 'uri'])  # Select Table attributes to export in dictionary keys

{'parameter': 'SENSOR',
 'date': Timestamp('2025-08-29 03:00:01'),
 'uri': 'http://vocab.nerc.ac.uk/collection/R25/current/'}

In [14]:
# Export table values to a pd.DataFrame:
art.to_dataframe()

,name,reference,long_name,definition,deprecated,version,date,uri,urn,parameter,related,broader,narrower,sameas,context
0,ACOUSTIC,R25,Acoustic sensor,Acoustic sensor,False,1,2019-10-11 15:06:36,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::ACOUSTIC,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
1,ACOUSTIC_GEOLOCATION,R25,Acoustic sensor measuring geolocation,Acoustic sensor for geolocation,False,1,2019-10-11 15:06:36,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::ACOUSTIC_GEOLOCATION,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
2,BACKSCATTERINGMETER_BBP470,R25,Backscattering meter measuring backscattering ...,Backscattering meter measuring backscattering ...,False,1,2021-02-09 15:34:44,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::BACKSCATTERINGMETER_BBP470,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
3,BACKSCATTERINGMETER_BBP532,R25,Backscattering meter measuring backscattering ...,Backscattering meter measuring backscattering ...,False,1,2021-02-09 15:34:44,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::BACKSCATTERINGMETER_BBP532,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
4,BACKSCATTERINGMETER_BBP700,R25,Backscattering meter measuring backscattering ...,Backscattering meter measuring backscattering ...,False,1,2021-02-09 15:34:44,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::BACKSCATTERINGMETER_BBP700,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
5,BACKSCATTERINGMETER_TURBIDITY,R25,Backscattering meter measuring turbidity,Backscattering meter measuring turbidity,False,1,2019-10-11 15:06:36,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::BACKSCATTERINGMETER_TURBIDITY,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
6,CTD_CNDC,R25,Conductivity Temperature Depth (CTD) sensors p...,Conductivity Temperature Depth (CTD) sensors p...,False,1,2019-10-11 15:06:36,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::CTD_CNDC,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
7,CTD_PRES,R25,Conductivity Temperature Depth (CTD) sensors p...,Conductivity Temperature Depth (CTD) sensors p...,False,1,2019-10-11 15:06:36,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::CTD_PRES,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
8,CTD_TEMP,R25,Conductivity Temperature Depth (CTD) sensors p...,Conductivity Temperature Depth (CTD) sensors p...,False,1,2019-10-11 15:06:36,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::CTD_TEMP,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
9,CTD_TEMP_CNDC,R25,Temperature sensor for conductivity on a CTD,Specific temperature sensor used with conducti...,False,1,2023-11-24 09:35:13,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::CTD_TEMP_CNDC,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None


In [15]:
art.to_dataframe(columns=['name', 'deprecated'])  # Select value attributes to export in columns

,name,deprecated
0,ACOUSTIC,False
1,ACOUSTIC_GEOLOCATION,False
2,BACKSCATTERINGMETER_BBP470,False
3,BACKSCATTERINGMETER_BBP532,False
4,BACKSCATTERINGMETER_BBP700,False
5,BACKSCATTERINGMETER_TURBIDITY,False
6,CTD_CNDC,False
7,CTD_PRES,False
8,CTD_TEMP,False
9,CTD_TEMP_CNDC,False


In [16]:
# Export table values to a dictionary, use pd.DataFrame:
art.to_dataframe(columns=['name', 'deprecated']).to_dict(orient='records')

[{'name': 'ACOUSTIC', 'deprecated': False},
 {'name': 'ACOUSTIC_GEOLOCATION', 'deprecated': False},
 {'name': 'BACKSCATTERINGMETER_BBP470', 'deprecated': False},
 {'name': 'BACKSCATTERINGMETER_BBP532', 'deprecated': False},
 {'name': 'BACKSCATTERINGMETER_BBP700', 'deprecated': False},
 {'name': 'BACKSCATTERINGMETER_TURBIDITY', 'deprecated': False},
 {'name': 'CTD_CNDC', 'deprecated': False},
 {'name': 'CTD_PRES', 'deprecated': False},
 {'name': 'CTD_TEMP', 'deprecated': False},
 {'name': 'CTD_TEMP_CNDC', 'deprecated': False},
 {'name': 'EM', 'deprecated': False},
 {'name': 'FLOATCLOCK_MTIME', 'deprecated': False},
 {'name': 'FLUOROMETER_CDOM', 'deprecated': False},
 {'name': 'FLUOROMETER_CHLA', 'deprecated': False},
 {'name': 'IDO_DOXY', 'deprecated': False},
 {'name': 'OPTODE_DOXY', 'deprecated': False},
 {'name': 'RADIOMETER_DOWN_IRR', 'deprecated': False},
 {'name': 'RADIOMETER_DOWN_IRR380', 'deprecated': False},
 {'name': 'RADIOMETER_DOWN_IRR412', 'deprecated': False},
 {'name': 'R

### Search method

In [17]:
art = ArgoReferenceTable('SENSOR_MODEL')
art

<argo.reference.table> 'R27'/'SENSOR_MODEL'
long_name: "Argo sensor models"
description: "Terms listing models of sensors mounted on Argo floats. Note: avoid using the manufacturer name and sensor firmware version in new entries when possible. Argo netCDF variable SENSOR_MODEL is populated by R27 altLabel."
uri: http://vocab.nerc.ac.uk/collection/R27/current/
version: 23 (2025-11-19 02:00:00)
keys[137]: 'AANDERAA_OPTODE', 'AANDERAA_OPTODE_3830', 'AANDERAA_OPTODE_3835', 'AANDERAA_OPTODE_3930', 'AANDERAA_OPTODE_4330', 'AANDERAA_OPTODE_4330F', 'AANDERAA_OPTODE_4831', 'AANDERAA_OPTODE_4831F', 'AMETEK', 'AMETEK_3000PSIA' and more ...

In [18]:
# Search methods (return a list of :class:`ArgoReferenceValue` with match):
# Any of the :class:`ArgoReferenceValue` attribute can be searched
art.search(name='RAMSES')         # Search in values name

[<argo.reference.table.value> 'RAMSES_ACC'
 long_name: "TriOS RAMSES ACC spectral imaging radiometer"
 definition: "A spectral imaging radiometer with cosine detector which measures radiance, irradiance or scalar irradiance in the Ultraviolet (UV), Vis (Visible) and Ultraviolet-visible (UV-Vis) ranges, manufactured by TriOS. Number of channels: 256."
 urn: "SDN:R27::RAMSES_ACC"
 uri: http://vocab.nerc.ac.uk/collection/R27/current/RAMSES_ACC/
 version: 1 (2021-07-01 16:48:21)
 deprecated: False
 reference/parameter: R27/SENSOR_MODEL
 relations[3]:
   - 1 "broader" value: 'R26/TRIOS'
   - "related" to 2 values : 'R25/RADIOMETER_PAR' and 'R25/RADIOMETER_DOWN_IRR'
 context: (not loaded yet, use key indexing to load)
 extra: (no extra attributes from definition string)]

In [19]:
art.search(definition='imaging')  # Search in values definition

[<argo.reference.table.value> 'RAMSES_ACC'
 long_name: "TriOS RAMSES ACC spectral imaging radiometer"
 definition: "A spectral imaging radiometer with cosine detector which measures radiance, irradiance or scalar irradiance in the Ultraviolet (UV), Vis (Visible) and Ultraviolet-visible (UV-Vis) ranges, manufactured by TriOS. Number of channels: 256."
 urn: "SDN:R27::RAMSES_ACC"
 uri: http://vocab.nerc.ac.uk/collection/R27/current/RAMSES_ACC/
 version: 1 (2021-07-01 16:48:21)
 deprecated: False
 reference/parameter: R27/SENSOR_MODEL
 relations[3]:
   - 1 "broader" value: 'R26/TRIOS'
   - "related" to 2 values : 'R25/RADIOMETER_PAR' and 'R25/RADIOMETER_DOWN_IRR'
 context: (not loaded yet, use key indexing to load)
 extra: (no extra attributes from definition string)]

In [20]:
art.search(long_name='TriOS')     # Search in values long name

[<argo.reference.table.value> 'OPUS_DS'
 long_name: "TriOS OPUS spectral sensor - Deep Sea version"
 definition: "A spectral sensor designed to measure nitrogen and carbon compounds, with titanium housing and depth-rating of 6000 dbar (Deep-Sea version). A xenon flash lamp provides the light source, and a 256-channel spectrometer covers a wavelength range of 200 nm - 360 nm. The instrument is manufactured by TriOS."
 urn: "SDN:R27::OPUS_DS"
 uri: http://vocab.nerc.ac.uk/collection/R27/current/OPUS_DS/
 version: 1 (2021-08-18 14:32:12)
 deprecated: False
 reference/parameter: R27/SENSOR_MODEL
 relations[3]:
   - 1 "broader" value: 'R26/TRIOS'
   - "related" to 2 values : 'R25/SPECTROPHOTOMETER_NITRATE' and 'R25/SPECTROPHOTOMETER_BISULFIDE'
 context: (not loaded yet, use key indexing to load)
 extra: (no extra attributes from definition string),
 <argo.reference.table.value> 'RAMSES_ACC'
 long_name: "TriOS RAMSES ACC spectral imaging radiometer"
 definition: "A spectral imaging radiomete

In [21]:
# Possible change to output format:
art.search(long_name='TriOS', output='df')  # To a :class:`pd.DataFrame`

,name,reference,long_name,definition,deprecated,version,date,uri,urn,parameter,related,broader,narrower,sameas,context
0,OPUS_DS,R27,TriOS OPUS spectral sensor - Deep Sea version,A spectral sensor designed to measure nitrogen...,False,1,2021-08-18 14:32:12,http://vocab.nerc.ac.uk/collection/R27/current...,SDN:R27::OPUS_DS,SENSOR_MODEL,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None
1,RAMSES_ACC,R27,TriOS RAMSES ACC spectral imaging radiometer,A spectral imaging radiometer with cosine dete...,False,1,2021-07-01 16:48:21,http://vocab.nerc.ac.uk/collection/R27/current...,SDN:R27::RAMSES_ACC,SENSOR_MODEL,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None


## ArgoReferenceValue

Rq: I hesitated to call this class ``ArgoReferenceConcept`` or ``ArgoVocabularyConcept``. But since the Argo user's manual uses "Reference Table", I opted to keep this approach and do not stick too much to the NVS jargon (which is still visible in the internal machinery though).

The ``ArgoReferenceValue`` instance holds all the Argo referencing system information:

- the comprehensive logic (e.g. the reference table is automatically determined when possible, hints are return otherwise),
- the value meta-data are in read-only attributes, e.g. ``arv.definition`` and ``arv.version``, ``arv.deprecated``, etc..., 
- value meta-data export methods, e.g. ``arv.to_dict()``, ``arv.to_json()``.

### Creation

In [22]:
from argopy import ArgoReferenceValue

# One possible value for the Argo parameter 'SENSOR_MODEL':
arv = ArgoReferenceValue('BBP700')

# For ambiguous value seen in more than one Reference Table
arv = ArgoReferenceValue('4', reference='RT_QC_FLAG')
arv = ArgoReferenceValue('4', reference='RR2')

# From NVS/URN jargon:
arv = ArgoReferenceValue.from_urn('SDN:R03::BBP700')

arv

<argo.reference.table.value> 'BBP700'
long_name: "Particle backscattering at 700 nanometers"
definition: "Particle backscattering at 700 nm wavelength, reported by ECO3 sensor. Local_Attributes:{long_name:Particle backscattering at 700 nanometers; standard_name:-; units:m-1; valid_min:-; valid_max:-; fill_value:99999.f}. Properties:{category:b; data_type:float}"
urn: "SDN:R03::BBP700"
uri: http://vocab.nerc.ac.uk/collection/R03/current/BBP700/
version: 3 (2025-11-25 10:13:59)
deprecated: False
reference/parameter: R03/PARAMETER
relations[2]:
  - "related" to 1 value : 'P06/UPRM'
  - "sameas" 1 value : 'P01/BB117NIR'
context[21]: 'cpm', 'qudt', 'void', 'pav', 'owl', 'dce', 'skos', 'ns0', 'rdfs', 'iop', 'ns1', 'puv', 'semapv', 'dct', 'rdf', 'reg', 'sssom', 'dcat', 'grg', 'prov' and 'dc'
extra[2]:
  - "Local_Attributes": LocalAttributes(long_name='Particle backscattering at 700 nanometers', standard_name='-', units='m-1', valid_min='-', valid_max='-', fill_value='99999.f')
  - "Properties

### Attributes

In [23]:
arv = ArgoReferenceValue('BBP700')

In [24]:
# All possible attributes are listed in:
arv.attrs

('name',
 'reference',
 'long_name',
 'definition',
 'deprecated',
 'version',
 'date',
 'uri',
 'urn',
 'parameter',
 'related',
 'broader',
 'narrower',
 'sameas',
 'context',
 'extra',
 'nvs')

In [25]:
# Reference Value attributes:
arv.name       # Term-id of the URN, eg 'AANDERAA_OPTODE_3835'
arv.long_name  # nvs["skos:prefLabel"]["@value"]
arv.definition # nvs["skos:definition"]["@value"]
arv.deprecated # nvs["owl:deprecated"]
arv.parameter  # The netcdf parameter this concept applies to (eg 'SENSOR_MODEL')
arv.reference  # The reference table this concept belongs to, can be used on a ArgoReferenceTable (eg 'R27')

'R03'

In [26]:
# Other reference Value attributes (more technical):
arv.version    # nvs["owl:versionInfo"]
arv.date       # nvs["dc:date"]
arv.uri        # nvs["@id"]
arv.urn        # nvs["skos:notation"]

'SDN:R03::BBP700'

In [27]:
# Relationships with other Reference Values or Context:
arv.broader    # nvs["skos:broader"]
arv.narrower   # nvs["skos:narrower"]
arv.related    # nvs["skos:related"]
arv.sameas     # nvs["owl:sameAs"]
arv.context    # nvs["@context"]

{'cpm': 'http://purl.org/voc/cpm#',
 'qudt': 'https://qudt.org/2.1/schema/qudt#',
 'void': 'http://rdfs.org/ns/void#',
 'pav': 'http://purl.org/pav/',
 'owl': 'http://www.w3.org/2002/07/owl#',
 'dce': 'http://purl.org/dc/elements/1.1/',
 'skos': 'http://www.w3.org/2004/02/skos/core#',
 'ns0': 'http://www.w3.org/ns/dx/conneg/altr#',
 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
 'iop': 'https://w3id.org/iadopt/ont#',
 'ns1': 'http://www.w3.org/ns/dx/prof/',
 'puv': 'https://w3id.org/env/puv#',
 'semapv': 'http://w3id.org/semapv/vocab/',
 'dct': 'http://purl.org/dc/terms/',
 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
 'reg': 'http://purl.org/linked-data/registry#',
 'sssom': 'https://w3id.org/sssom/schema/',
 'dcat': 'http://www.w3.org/ns/dcat#',
 'grg': 'http://www.isotc211.org/schemas/grg/',
 'prov': 'https://www.w3.org/ns/prov#',
 'dc': 'http://purl.org/dc/terms/'}

In [28]:
# Raw NVS json data:
arv.nvs.keys()

dict_keys(['@id', 'pav:hasVersion', 'pav:authoredOn', '@type', 'skos:related', 'skos:definition', 'void:inDataset', 'owl:deprecated', 'skos:note', 'dc:identifier', 'skos:prefLabel', 'owl:versionInfo', 'dc:date', 'skos:altLabel', 'skos:notation', 'owl:sameAs', 'pav:version', 'dce:identifier', 'pav:hasCurrentVersion', '@context'])

### Extra attributes (only for values from R03, R14, R18)

In [29]:
# For Values from R03 table
arv = ArgoReferenceValue('BBP470')

arv.extra

{'Local_Attributes': LocalAttributes(long_name='Particle backscattering at 470 nanometers', standard_name='-', units='m-1', valid_min='-', valid_max='-', fill_value='99999.f'),
 'Properties': Properties(category='b', data_type='float', extra_dim=None)}

In [30]:
arv.extra['Local_Attributes'].long_name

'Particle backscattering at 470 nanometers'

In [31]:
arv.extra['Properties'].category

'b'

In [32]:
# For Values from R14 table
arv = ArgoReferenceValue('T000015')
arv.extra

{'Template_Values': TemplateValues(unit=['degC', 'mdegC'])}

In [33]:
arv.extra['Template_Values'].unit

['degC', 'mdegC']

In [34]:
# For Values from R18 table
arv = ArgoReferenceValue('CB00001')
arv.extra

{'Template_Values': TemplateValues(short_sensor_name=['Crover', 'Ctd', 'Eco', 'Flbb', 'Flntu', 'Ocr', 'Optode', 'Sfet', 'Suna'], N='[1..4]', unit=['bar', 'dbar', 'cbar', 'mbar', 'inHg'])}

In [35]:
arv.extra['Template_Values'].short_sensor_name

['Crover', 'Ctd', 'Eco', 'Flbb', 'Flntu', 'Ocr', 'Optode', 'Sfet', 'Suna']

### Export methods

In [36]:
arv = ArgoReferenceValue('AANDERAA_OPTODE_3835')

# Export to a dictionary:
arv.to_dict()

{'name': 'AANDERAA_OPTODE_3835',
 'reference': 'R27',
 'long_name': 'Aanderaa 3835 oxygen optode',
 'definition': 'A stand-alone oxygen optode with integrated temperature sensor, manufactured by Aanderaa. This instrument exploits the physio-chemical principle of dynamic fluorescence quenching to measure absolute oxygen concentration and percent saturation. Depth rating: 300 m; oxygen concentration accuracy of < 8 uM or 5 %, and resolution of < 1 uM; air saturation accuracy of < 5 %, and resolution of 0.4 %; settling time (63 %): < 25 s.',
 'deprecated': False,
 'version': '1',
 'date': Timestamp('2021-05-26 09:01:46'),
 'uri': 'http://vocab.nerc.ac.uk/collection/R27/current/AANDERAA_OPTODE_3835/',
 'urn': 'SDN:R27::AANDERAA_OPTODE_3835',
 'parameter': 'SENSOR_MODEL',
 'related': [{'@id': 'http://vocab.nerc.ac.uk/collection/R25/current/OPTODE_DOXY/'}],
 'broader': [{'@id': 'http://vocab.nerc.ac.uk/collection/R26/current/AANDERAA/'}],
 'narrower': None,
 'sameas': None,
 'context': {'cpm

In [37]:
arv.to_dict(keys=['name', 'deprecated'])  # Select attributes to export in dictionary keys

{'name': 'AANDERAA_OPTODE_3835', 'deprecated': False}

In [38]:
# Export to json structure:
arv.to_json()  # In memory
# arv.to_json('reference_value.json')  # To a json file
# arv.to_json('reference_value.json', keys=['name', 'deprecated'])  # Select attributes to export

'{"name": "AANDERAA_OPTODE_3835", "reference": "R27", "long_name": "Aanderaa 3835 oxygen optode", "definition": "A stand-alone oxygen optode with integrated temperature sensor, manufactured by Aanderaa. This instrument exploits the physio-chemical principle of dynamic fluorescence quenching to measure absolute oxygen concentration and percent saturation. Depth rating: 300 m; oxygen concentration accuracy of < 8 uM or 5 %, and resolution of < 1 uM; air saturation accuracy of < 5 %, and resolution of 0.4 %; settling time (63 %): < 25 s.", "deprecated": false, "version": "1", "date": "2021-05-26 09:01:46", "uri": "http://vocab.nerc.ac.uk/collection/R27/current/AANDERAA_OPTODE_3835/", "urn": "SDN:R27::AANDERAA_OPTODE_3835", "parameter": "SENSOR_MODEL", "related": [{"@id": "http://vocab.nerc.ac.uk/collection/R25/current/OPTODE_DOXY/"}], "broader": [{"@id": "http://vocab.nerc.ac.uk/collection/R26/current/AANDERAA/"}], "narrower": null, "sameas": null, "context": {"cpm": "http://purl.org/voc/

## ArgoReferenceMapping

A mapping is a list of relationships between 1 concept (the 'subject') and others (the 'object'). Subjects are grouped by reference tables, as well as subjects. So that there will be one *mapping* between one ArgoReferenceTable and another.

Relationships between concepts can be: 
- "narrower/broader" when there is a hierarchy between the subject and the object
- "related" or "sameas" when the subject is related to the object without strict hierarchy

In this high-level API I opted to remove the NVS jargon of 'skos' and 'owl', to simplify relationships understanding. But in order to avoid too much confusion for the AVTT, I also opted to keep "predicates" to refer to relationships. **May be this is not the best choice and 'relations' could be preferred over 'predicates'**.

**List of known mappings**:

|Subject|Predicate|Object|Dedicated Argopy support |
|---|---|---|----------------------|
|**R08** Argo instrument types (eg: '878')| broader |**R23** Argo platform type (eg: 'ARVOR') | ❌                    |
|**R24** Argo platform maker (eg: 'MRV')|related|**R23** Argo platform type (eg: 'SOLO_D_MRV')|❌                    |
|**RMC** Argo measurement code categories (eg: 'RSPEC')| narrower | **R15** Argo trajectory measurement code identifiers (eg: '901')  | ❌                    |
|**RTV** Argo float cycle timing variables (eg: 'DET')| related or sameas | **R15** Argo trajectory measurement code identifiers (eg: '901')  | ❌                    |
|**R25** Argo sensor types (eg: 'RADIOMETER_DOWN_IRR380')|related|**R27** Argo sensor models (eg: 'SATLANTIC_OCR504_ICSW')| ✅  in the ``ArgoSensor`` [sub-module still in dev.](https://github.com/euroargodev/argopy/pull/572)                   |
|**R26** Argo sensor manufacturers (eg: 'SBE')|narrower|**R27** Argo sensor models (eg: 'SEAFET')| ❌                    |


Eg:
- R23/APEX is a broader concept than R08/847
- R27/SEAFET is a narrower concepth than R26/SBE
- R27/AANDERAA_OPTODE_3830 is related to R25/OPTODE_DOXY
- R27/AANDERAA_OPTODE_3830 is a broader concept than R26/AANDERAA



More details from the AVTT documentation:
https://github.com/OneArgo/ArgoVocabs?tab=readme-ov-file#ivb-mappings

> Mappings are used to inform relationship between concepts. For instance, inform all the sensor_models manufactured by one sensor_maker, or all the platform_types manufactures by one platform_maker, etc.
> They are used by the FileChecker to ensure the consistency between these metadata fields in the Argo dataset.

### Creation

In [39]:
from argopy import ArgoReferenceMapping

# Use two Argo parameter names, documented by one of the Argo reference tables:
ArgoReferenceMapping('PLATFORM_MAKER', 'PLATFORM_TYPE')

# or reference table identifiers:
ArgoReferenceMapping('R24', 'R23')

<argo.reference.mapping> subject('R24'/'PLATFORM_MAKER') vs object('R23'/'PLATFORM_TYPE')
44 relationships in this mapping

### Indexing and values

In [40]:
arm = ArgoReferenceMapping('R24', 'R23')

In [41]:
# Relationships within this reference mapping:
len(arm)     # Number of relationships
arm.subjects   # Ordered list of unique 'subject' reference values names
arm.objects    # Ordered list of unique 'object' reference values names
arm.predicates # Ordered list of unique 'predicate', aka relationships, in this mapping

['related']

In [42]:
# Check if a reference value is in this mapping as a subject or an object:
'SBE' in arm  # Return True

True

In [43]:
# Indexing is by subject values:
arm['SBE']  # Return a dict with predicate as keys and objects as values

{'related': ['NAVIS_A', 'NAVIS_EBR']}

In [44]:
# Iterate over all relationships:
for relation in arm:
    print(relation['subject'], relation['predicate'])

HSOE {'related': ['HM2000']}
LaoshanL {'related': ['HM4000', 'XUANWU']}
MARTEC {'related': ['PROVOR', 'PROVOR_II']}
METOCEAN {'related': ['NOVA', 'PROVOR_MT']}
MRV {'related': ['ALAMO', 'ALTO', 'S2A', 'S2X', 'SOLO_BGC_MRV', 'SOLO_D_MRV']}
NKE {'related': ['ARVOR', 'ARVOR_C', 'ARVOR_D', 'POPS_PROVOR', 'PROVOR', 'PROVOR_II', 'PROVOR_III', 'PROVOR_IV', 'PROVOR_V', 'PROVOR_V_JUMBO']}
NOTC {'related': ['COPEX']}
OPTIMARE {'related': ['NEMO', 'POPS_NEMO']}
QNLM {'related': ['HM4000', 'XUANWU']}
SBE {'related': ['NAVIS_A', 'NAVIS_EBR']}
SIO_IDG {'related': ['SOLO', 'SOLO_BGC', 'SOLO_D', 'SOLO_II']}
TSK {'related': ['NINJA', 'NINJA_D']}
TWR {'related': ['APEX', 'APEX_D', 'APEX_EM']}
WHOI {'related': ['ITP', 'SOLO_W']}
WRC {'related': ['APEX', 'APEX_EM', 'PALACE']}


### Export method

In [45]:
arm = ArgoReferenceMapping('R24', 'R23')

In [46]:
# Export all mapping relationships in a DataFrame:
arm.to_dataframe()

,subject,predicate,object
0,HSOE,related,HM2000
1,LaoshanL,related,HM4000
2,LaoshanL,related,XUANWU
3,MARTEC,related,PROVOR
4,MARTEC,related,PROVOR_II
5,METOCEAN,related,NOVA
6,METOCEAN,related,PROVOR_MT
7,MRV,related,ALAMO
8,MRV,related,ALTO
9,MRV,related,S2A


In [47]:
# To export mapping using AVTT jargon:
arm.to_dataframe(raw=True)

,subject,predicate,object
0,http://vocab.nerc.ac.uk/collection/R24/current...,skos:related,http://vocab.nerc.ac.uk/collection/R23/current...
1,http://vocab.nerc.ac.uk/collection/R24/current...,skos:related,http://vocab.nerc.ac.uk/collection/R23/current...
2,http://vocab.nerc.ac.uk/collection/R24/current...,skos:related,http://vocab.nerc.ac.uk/collection/R23/current...
3,http://vocab.nerc.ac.uk/collection/R24/current...,skos:related,http://vocab.nerc.ac.uk/collection/R23/current...
4,http://vocab.nerc.ac.uk/collection/R24/current...,skos:related,http://vocab.nerc.ac.uk/collection/R23/current...
5,http://vocab.nerc.ac.uk/collection/R24/current...,skos:related,http://vocab.nerc.ac.uk/collection/R23/current...
6,http://vocab.nerc.ac.uk/collection/R24/current...,skos:related,http://vocab.nerc.ac.uk/collection/R23/current...
7,http://vocab.nerc.ac.uk/collection/R24/current...,skos:related,http://vocab.nerc.ac.uk/collection/R23/current...
8,http://vocab.nerc.ac.uk/collection/R24/current...,skos:related,http://vocab.nerc.ac.uk/collection/R23/current...
9,http://vocab.nerc.ac.uk/collection/R24/current...,skos:related,http://vocab.nerc.ac.uk/collection/R23/current...
